

> create database in beaver (from empty database)

postgres - sql editor - open SQL script


In [ ]:
create database new_vip_orders_database 

play script and refresh postgres

click to the our new created database - tools - execute scripts - input file (choose path to database) - start

In [ ]:
select * from expensive_expenses ee 

In [ ]:
select id, count(id), sum(cost)
from expensive_purchases ep 
where payment_method ='cash'
group by id
order by sum(cost) desc 
limit 10

In [ ]:
select * 
from 
(select 
    extract (year from date) as year,
    case 
        when extract (month from date) in (3,4,5) then 'spring'
        when extract (month from date) in (9,10,11) then 'autumn'
    end as seasons,
    avg(cost)
from expensive_purchases ep
group by year, seasons) as q_l
where seasons is not null

In [ ]:
select product, 
       cost 
from expensive_purchases 
where id in (select id
    from expensive_purchases
    group by id 
    having min(date) between '2019-03-01' and '2019-03-31')
order by cost desc 
limit 1

In [ ]:
SELECT
    order_details.product_id,
    order_details.order_id,
    products.supplier_id,
    suppliers.company_name
FROM
    order_details
    LEFT JOIN products ON  order_details.product_id = products.product_id
    LEFT join suppliers on suppliers.supplier_id = products.supplier_id
order by 
    

In [ ]:
### 1st homework 1st task
select suppliers.company_name as supplier_name, count(order_details.quantity) as total_orders 
FROM
    order_details
    inner JOIN products ON  order_details.product_id = products.product_id
    inner join suppliers on suppliers.supplier_id = products.supplier_id
group by supplier_name
order by total_orders desc limit 10

In [ ]:
### another solution ****
select s.company_name as supplier_name, count(od.product_id) as total_orders 
FROM
    order_details od
    left outer JOIN products as p ON  od.product_id = p.product_id
    left outer join suppliers as s on p.supplier_id = s.supplier_id
group by s.company_name
order by total_orders desc 
limit 10

In [ ]:
### 1st homework 2nd task
select 
    extract (month from order_date) as order_month,
    count(customer_id) as total_customers
from orders
where customer_id in (select customer_id
    from orders
    group by customer_id
    having min(order_date) between '1997-01-01' and '1997-01-31') and 
    order_date between '1997-01-01' and '1997-12-31'
group by order_month

In [ ]:
### 1st homework 3nd task
select round(sum(unit_price*quantity)) 
from order_details inner join orders on orders.order_id = order_details.order_id
where customer_id in (select customer_id
    from orders
    where customer_id in 
    (select customer_id from orders where extract(year from order_date)='1998')
    group by customer_id
    having min(order_date) between '1997-09-01' and '1997-09-30')

In [ ]:
### 2nd webinar

In [ ]:
update expensive_purchases 
set product = 'collimator'
where product = 'optical'

In [ ]:
update expensive_purchases 
set cost = cost/10,
    payment_method = 'cash' 
where product = 'rifle' 
and payment_method = 'cash'

In [ ]:
delete from expensive_purchases 
where product = 'watch'

In [ ]:
select category
from expensive_expenses ee 
group by category
order by count(expenses)
limit 1

In [ ]:
delete from expensive_expenses 
where category = (
	select category
	from expensive_expenses ee 
	group by category
	order by count(expenses)
	limit 1
				)

In [ ]:
insert into expensive_expenses (expenses_id, category, date, expenses)
values
(299,'electronics', '2019-03-21', 4532),
(300, 'hunting', '2020-03-30', 1890),
(301, 'transport', '2019-12-12', 4444)

In [ ]:
# second homework task 1
select * from customer_demographics cd 
INSERT INTO customer_demographics (customer_type_id, customer_desc)
SELECT customer_id, country
FROM customers

In [ ]:
# second homework task 2
delete from shippers s 
where shipper_id not in
(
select ship_via
from orders
group by ship_via 
)

In [ ]:
# second homework task 3
delete from products p 
where unit_price not between 
	(select 
	    percentile_disc(0.05) within group (order by products.unit_price) ---  между 0,05 (а не 0.5) и 0,95 квантилью
	from products)
and 
	(select 
	    percentile_disc(0.95) within group (order by products.unit_price)
	from products)
 
 ###

delete from order_details where product_id in
(select product_id from products p 
where unit_price not between 
	(select 
	    percentile_disc(0.05) within group (order by products.unit_price) ---  между 0,05 (а не 0.5) и 0,95 квантилью
	from products)
and 
	(select 
	    percentile_disc(0.95) within group (order by products.unit_price)
	from products))


In [ ]:
# another solution for 3
--delete from order_details where product_id in
delete from products  where product_id in(
with quantily as (
    select percentile_disc(0.5)  within group (order by unit_price) as low,
           percentile_disc(0.95) within group (order by unit_price) as top
      from products
)
select product_id
  from products, quantily  
 --where unit_price between quantily.low and quantily.top;
 where unit_price < quantily.low or unit_price > quantily.top);

In [ ]:
# 3 webinar

# Задача 1 Создать таблицу для связей между двумя таблицами.
CREATE TABLE category_product
(
product_id VARCHAR(25) NOT NULL,
PRIMARY KEY (product_id)
);


In [ ]:
# Задача 2 Заполнить первое поле таблицы category_product (созданной только что, поле product_id)
INSERT INTO category_product (product_id)
(SELECT distinct (product)
FROM expensive_purchases)

In [ ]:
# Задача 3 Создать два дополнительных столбца.
ALTER TABLE category_product
ADD category_id VARCHAR(25),
ADD temp_id VARCHAR(25)

# Задача 4 Переименовать столбец temp_id в temp_1.
ALTER TABLE category_product
RENAME COLUMN temp_id TO temp_1;
ALTER TABLE category_product
ALTER COLUMN temp_1 TYPE VARCHAR(5)

# Задача 5 Удалить temp_1.
ALTER TABLE category_product
DROP COLUMN temp_1

In [ ]:
# Задача 6 Заполнить второе поле (category_id)
SELECT distinct category
FROM expensive_expenses;
UPDATE category_product
SET category_id =
(CASE
WHEN product_id IN ('auto', 'motorcycle', 'bicycle') THEN 'transport'
WHEN product_id IN ('collimator', 'knife', 'rifle') THEN 'hunting'
ELSE 'electronics'
END)

# Задача 6 Сделать связь для category_product и
ALTER TABLE expensive_purchases
ADD FOREIGN KEY (product) REFERENCES category_product (product_id);